# Movie Dataset Cleaning

First, we will keep our focus on cleaning the "movie.metadata" dataset. 
The goal is to do a deep review of the whole dataset, have a good understanding of the missing data for each relevant feature for our study and to have a cleaned version ready for Milestone 3.

### Loading the Dataset

In [126]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from importlib import reload
import src.utils.utils
reload(src.utils.utils)
from src.utils.utils import extract_info, exploded_format, dropempty

In [127]:
folder = '../data/CMU/'
pickle_folder = "../data/pickle/"

In [128]:
movie_original_data = pd.read_csv(folder + 'movie.metadata.tsv' ,sep='\t',names=['Wikipedia_movie_ID',
'Freebase_movie_ID',
'Movie_name',
'Movie_release_date',
'Movie_box_office_revenue',
'Movie_runtime',
'Movie_languages_(Freebase_ID:name_tuples)',
'Movie_countries_(Freebase_ID:name_tuples)',
'Movie_genres_(Freebase_ID:name_tuples)'])

In [129]:
movies = movie_original_data.copy()

In [130]:
print("The original number of movie in movie.metadata dataset is : {}".format(movies.shape[0]))

The original number of movie in movie.metadata dataset is : 81741


## 1. Dropping invalid values for Movie box office revenue

- Most important, because our main question is focusing on the implications of characteristics on the overall box office performance of movies, the first goal is to drop all the movie lines where the box office is not detailed. 

In [131]:
movies_with_box_office = movies.dropna(subset=['Movie_box_office_revenue'])

In [132]:
print("The number of movie with a valid value for the box office revenue in movie.metadata dataset :\n {}".format(movies_with_box_office.shape[0]))

The number of movie with a valid value for the box office revenue in movie.metadata dataset :
 8401


- We notice that dropping all the movies without a valid box office revenue value reduces the size of the dataframe by a factor of almost 10. Hence, we think that we should take another criteria to complete the evaluation of the success of movies. That was the main reason that pushed us to study the imdb dataset and its ratings.

##  2. Cleaning Features

- Before starting the analysis of the features relevant to our project, we will drop the columns that are irrelevant to us or that we don’t intend to use, to avoid being overwhelmed by unnecessary information when printing the data frames.

In [133]:
movies_clean = movies.drop(columns=['Freebase_movie_ID','Movie_runtime'])
movies_clean.sample(5)

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_languages_(Freebase_ID:name_tuples),Movie_countries_(Freebase_ID:name_tuples),Movie_genres_(Freebase_ID:name_tuples)
66763,5114762,Code Breakers,2006-05-10,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/0jtdp"": ""Docum..."
18729,13569878,Toy Soldiers,1984,NaN,"{""/m/02h40lc"": ""English Language"", ""/m/06nm1"":...","{""/m/0b90_r"": ""Mexico"", ""/m/09c7w0"": ""United S...","{""/m/01jfsb"": ""Thriller"", ""/m/03btsm8"": ""Actio..."
79241,8960083,Megaville,1990,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
71213,31322758,Heimliche Ehen,1956,NaN,"{""/m/04306rv"": ""German Language""}","{""/m/03f2w"": ""German Democratic Republic""}","{""/m/07s9rl0"": ""Drama""}"
6183,498211,Jesus of Montreal,1989,NaN,"{""/m/064_8sq"": ""French Language"", ""/m/02bjrlw""...","{""/m/0f8l9c"": ""France"", ""/m/0d060g"": ""Canada""}","{""/m/04p5fxn"": ""Christian film"", ""/m/07s9rl0"":..."


###  2.1. Cleaning the Dates

- First, we saw that for the movie "Hunting Season" the release date written was "1010-12-02" but the real release date is  "2010-12-02" :

In [134]:
movies_clean.loc[movies['Movie_name'] == 'Hunting Season']

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_languages_(Freebase_ID:name_tuples),Movie_countries_(Freebase_ID:name_tuples),Movie_genres_(Freebase_ID:name_tuples)
62836,29666067,Hunting Season,1010-12-02,12160978.0,"{""/m/02hwyss"": ""Turkish Language"", ""/m/02h40lc...","{""/m/01znc_"": ""Turkey""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/02n4kr"": ""My..."


Hence, we modified this value by the true one : 

In [135]:
movies_clean.loc[62836, 'Movie_release_date'] = '2010-12-02'
print(movies_clean.loc[62836, 'Movie_release_date'])

2010-12-02


- Drop all the movies where no release date are written

In [136]:
movies_clean = movies_clean.dropna(inplace = False, subset = 'Movie_release_date').copy()

In [137]:
movies_clean.shape[0]

74839

- Create a column 'Year' where we only have the released year of the movie (to do year by year analysis) : 

In [138]:
movies_clean['Year'] = movies_clean['Movie_release_date'].str[:4]
movies_clean['Year'] = movies_clean['Year'].astype(int)

- For our study, in addition to analyzing the dataset year by year, we decided to conduct an analysis across six different intervals of approximately 20 years each, spanning from 1915 to 2015. We decided to restrain ourself to that particular period because of the lack of meaningful data befor the 1910 decade.
- We created a 'Year_Interval' column in which each film is categorized into one of the six designated study intervals.
- Finally, we save our new version of the movies dataset with these new columns

In [155]:
movies_clean['Year_Interval'] = pd.cut(movies_clean['Year'], bins=[1914, 1935, 1955, 1975, 1995, 2016], labels=['1915-1935', '1935-1955', '1955-1975', '1975-1995', '1995-2015'])
movies_clean['Year_Interval'] = movies_clean['Year_Interval'].astype(str)

movies_clean = movies_clean.query(" 2016 > Year > 1914")

pickle.dump( movies_clean, open(pickle_folder + "movies_clean.p", "wb" ) )

- Now, because we will also do some analysis using the seasons (summer, winter, fall, spring) of movies' release date. We create another DataFrame, in which each movie with a specified release date is categorized by the season in which it was released.
- We also save this Dataframe creating a second version of our clean dataset. We prefer to have these rather than just merging the two and keeping only one because the creation of the 'release_season' excludes all the movies for which we don't have information about the month of release

In [140]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.utils.utils import categorize_release_season

In [141]:
md_release_season = movies_clean.copy()
md_release_season['Movie_release_date'] = pd.to_datetime(md_release_season['Movie_release_date'], errors='coerce')
md_release_season.dropna(subset = 'Movie_release_date', inplace = True)

In [142]:
md_release_season['release_season'] = md_release_season['Movie_release_date'].apply(categorize_release_season)
md_release_season.shape[0]

38531

In [143]:
md_release_season
pickle.dump( md_release_season, open(pickle_folder + "movies_clean_with_season.p", "wb" ) )

### 2.2. Clean 'Genres' Column 

- The ‘genres’ characteristic is among the key features to answer our general question. However, we noticed that in the movies.metadata dataset, this feature was presented in a (Freebase_ID:name_tuples) format, and thus wasn’t very clear and understandable. That is why we decided to reformat the ‘genres’ feature.
- We dropped all the movies that have no information about their genres (more than 1700)

In [144]:
column_name = 'Movie_genres_(Freebase_ID:name_tuples)' 

md_Genres = movies_clean.copy()
md_Genres["Genres"] = md_Genres[column_name].apply(extract_info)

md_Genres, n = dropempty(md_Genres,'Genres')
print(f"Number of movies dropped : {n}")

Number of movies dropped : 1756


- We splits the "Genres" column into a new DataFrame, expanding lists into separate columns. We calculates the number of genres for each entry and adds this count as a new column. As we can see when using the describe() method, each movie has 3 'Genres' in average.

In [145]:
genres_split = pd.DataFrame(md_Genres["Genres"].tolist(), index=md_Genres.index)
md_Genres['nb_of_Genres'] = md_Genres["Genres"].apply(lambda x:len(x))
md_Genres['nb_of_Genres'].describe()

count    71771.000000
mean         3.165931
std          2.112019
min          1.000000
25%          1.000000
50%          3.000000
75%          4.000000
max         17.000000
Name: nb_of_Genres, dtype: float64

In [146]:
# We keep only the first three genres of each row
genres_split = genres_split.iloc[:, :3]
genres_split = genres_split.add_prefix("Genres_")
genres_split.sample(5)

,Genres_0,Genres_1,Genres_2
31614,"""Crime Fiction""","""Gangster Film""","""Drama"""
77185,"""Thriller""","""Action""","""Adventure"""
15086,"""Thriller""","""Mystery""","""Romance Film"""
19969,"""Drama""",None,None
78615,"""Parody""","""Thriller""","""Comedy-drama"""


- We update the previous dataframes by adding the new 'genres' columns

In [147]:
data_genres = md_Genres.join(genres_split).drop(columns=["nb_of_Genres","Genres","Movie_genres_(Freebase_ID:name_tuples)"])

with open(pickle_folder + 'movies_clean.p', 'wb') as f:
    pickle.dump(data_genres, f)  
    
with open(pickle_folder + 'movies_clean_with_season.p', 'wb') as f:
    movie_clean_with_season = md_release_season.join(genres_split).drop(columns=["Movie_genres_(Freebase_ID:name_tuples)"])
    pickle.dump(movie_clean_with_season, f)

- We also create a dataframe containing 'genres' in an exploded format in order to facilitate the frequency analysis of genres

In [148]:
exploded_format('Genres',md_Genres,pickle_folder + "movies_genres_exploded.p")

Genres
"Drama"                31547
"Comedy"               15402
"Romance Film"          9730
"Thriller"              8396
"Action"                8266
                       ...  
"Statutory rape"           1
"Romantic thriller"        1
"Chick flick"              1
"Buddy Picture"            1
"Neorealism"               1
Name: count, Length: 363, dtype: int64

### 2.3. Clean 'Languages' Column

- In the same way as the part for Genres’ characteristics, this feature was presented in a (Freebase_ID:name_tuples) format, and thus wasn’t very clear and understandable. That is why we decided to reformat the ‘Language’ feature.
- We dropped all the movies that have no information about their languages (~ 9900)

In [149]:
column_name = 'Movie_languages_(Freebase_ID:name_tuples)'

md_language = movies_clean.copy()
md_language['Language'] = md_language[column_name].apply(extract_info)


md_language, n = dropempty(md_language,'Language')
print(f"Number of movies dropped : {n}")

Number of movies dropped : 9859


- We splits the "Language" column into a new DataFrame, expanding lists into separate columns. We calculate the number of languages for each entry and add this count as a new column. As we can see when using the describe() method, each movie has 1 "Language" on average.

In [150]:
language_split = pd.DataFrame(md_language["Language"].tolist(), index=md_language.index)
md_language['nb_of_Languages'] = md_language["Language"].apply(lambda x:len(x))
md_language['nb_of_Languages'].describe()

count    63668.000000
mean         1.195828
std          0.576698
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         15.000000
Name: nb_of_Languages, dtype: float64

- We decided not to save a split version of the dataframe with languages because of the inbalance between the mean and the max, resulting in 15 new columns almost filled of None. With the 'Genres' features we could select the top three genres to avoid that problem but we couldn't do the same for 'Language' feature as there is no order between the languages.

- We create a new dataframe containing the Languages in an exploded format in order to facilitate the frequency analysis of languages

In [151]:
exploded_format('Language',md_language,pickle_folder + "movies_languages_exploded.p")

Language
"English Language"           38867
"Spanish Language"            3526
"Hindi Language"              3431
"French Language"             3293
"Italian Language"            2434
                             ...  
"Sunda Language"                 1
"Hazaragi Language"              1
"Pawnee Language"                1
"Gumatj Language"                1
"Judeo-Georgian Language"        1
Name: count, Length: 191, dtype: int64

### 2.4. Clean 'Countries' Column 

- For the Countries feature, similar to the approach used for the Genres feature, the data was originally presented in a (Freebase_ID:name_tuples) format, making it unclear and difficult to interpret. To address this, we chose to reformat the Countries feature into a more understandable structure.
- We dropped all the movies that have no information about their countries (~ 5000)

In [152]:
column_name = "Movie_countries_(Freebase_ID:name_tuples)"

md_countries = movies_clean.copy()
md_countries["Countries"] = md_countries[column_name].apply(extract_info)

md_countries, n = dropempty(md_countries,'Countries')
print(f"Number of movies dropped : {n}")

Number of movies dropped : 5070


- We splits the "Countries" column into a new DataFrame, expanding lists into separate columns. We calculate the number of countries for each entry and add this count as a new column. As we can see when using the describe() method, each movie has 1 "Countries" on average.

In [153]:
countries_split = pd.DataFrame(md_countries["Countries"].tolist(), index=md_countries.index)
md_countries['nb_of_Countries'] = md_countries["Countries"].apply(lambda x:len(x))
md_countries['nb_of_Countries'].describe()

count    68457.000000
mean         1.182494
std          0.555381
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         14.000000
Name: nb_of_Countries, dtype: float64

- Again, very similar to the 'Language' feature, we decided not to save a split version of the dataframe because of the inbalance between the mean and the max. We also couldn't choose the top 1 country for each movie as there is no order between the countries listed for each movie.

- We create a new dataframe containing the Countries in an exploded format in order to facilitate the frequency analysis of countries

In [154]:
exploded_format('Countries',md_countries,pickle_folder + "movies_countries_exploded.p")

Countries
"United States of America"    32387
"India"                        7762
"United Kingdom"               7379
"France"                       4077
"Italy"                        3002
                              ...  
"Iraqi Kurdistan"                 1
"Jordan"                          1
"Macau"                           1
"Palestinian Territories"         1
"Republic of China"               1
Name: count, Length: 146, dtype: int64